In [1]:
# Libraries
import numpy as np
import pandas as pd
import re

In [2]:
# Importing 
attacks = pd.read_csv(r'../data/attacks.csv', encoding="Latin-1")

In [3]:
# Columns in the project
attacks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [4]:
# Dimension of the project 25723 Values (Rows), 24 Categories (Columns) 
attacks.shape

(25723, 24)

In [5]:
# I'm going to delete the columns with information im not going to use, Like "Case Number" "Case Number.1" and "Case Number.2" 
attacks = attacks.drop('Case Number.1', axis=1)
attacks = attacks.drop('Case Number.2', axis=1)
# I'm going to delete the columns with information that i dont't know what is inside because the header is unespecific
attacks = attacks.drop('Unnamed: 22', axis=1)
attacks = attacks.drop('Unnamed: 23', axis=1)
attacks = attacks.drop('Investigator or Source', axis=1)
attacks = attacks.drop('Species ', axis=1)

In [6]:
# I'm going to delete the columns with information that aren't usefull for me to use and is only noise.
attacks = attacks.drop('href formula', axis=1)
attacks = attacks.drop('href', axis=1)
attacks = attacks.drop('pdf', axis=1)
attacks = attacks.drop('original order', axis=1)
attacks = attacks.drop('Area', axis=1) # We already have Country
attacks = attacks.drop('Location', axis=1) # We already have Country
attacks = attacks.drop('Age', axis=1) # Dont gave information
attacks = attacks.drop('Name', axis=1) # Dont gave information

In [7]:
# We delete all ROWS full of NaN on it 
attacks = attacks[~attacks.drop('Case Number', axis=1).isna().all(axis=1)]

In [8]:
attacks = attacks.drop('Case Number', axis=1)

In [9]:
attacks.shape

(6302, 9)

In [10]:
header_mapping = {
    'Year': 'year',
    'Type': 'type',
    'Country' : 'country',
    'Activity' : 'activity',
    'Sex ' : 'sex',
    'Injury' : 'injury',
    'Fatal (Y/N)' : 'fatal_boolean',
    'Time' : 'time',
}
attacks = attacks.rename(columns=header_mapping)

In [11]:
attacks.dtypes

unique_values = attacks['fatal_boolean'].unique()
print(unique_values)

['N' 'Y' nan 'M' 'UNKNOWN' '2017' ' N' 'N ' 'y']



### 1st we want to change 'year'

In [12]:
# 1s we want to change 'year'
attacks['year'].fillna('unknwown', inplace=True)
attacks['year'] = attacks['year'].astype(str)

In [13]:

attacks['year'] = attacks['year'].astype(str).str.replace('u', 'unknwown')
attacks['year'] = attacks['year'].astype(str).str.replace(' ', 'unknwown')
attacks['year'] = attacks['year'].astype(str).str.replace('.0', 'unknwown')
attacks['year'] = attacks['year'].replace('77', 'unknwown', inplace=True)
attacks['year'] = attacks['year'].replace('500', 'unknwown', inplace=True)
attacks['year'] = attacks['year'].replace('5', 'unknwown', inplace=True)
attacks['year'] = attacks['year'].replace('0', 'unknwown', inplace=True)

### 2. Now we want to change 'type'

In [14]:
#2. Now we want to change 'type'
attacks['type'].fillna('UNKNOWN', inplace=True)
attacks['type'] = attacks['type'].astype(str)
attacks['type'] = attacks['type'].astype(str).str.replace('Boating', 'Boat',)
attacks['type'] = attacks['type'].astype(str).str.replace('Boatomg', 'Boat',)
attacks['type'] = attacks['type'].astype(str).str.replace('nan', 'Invalid',)

### 3. Country

In [15]:
# Now we want to change 'country', Latter we would create subgroups by continent to see they form part of each continent
attacks['country'].fillna('UNKNOWN', inplace=True)
attacks['country'] = attacks['country'].astype(str).str.replace('nan', 'UNKNOWN',)

### 4. Sex

In [16]:
#3. Let agroup sex
attacks['fatal_boolean'].fillna('UNKNOWN', inplace=True)
attacks['sex'] = attacks['sex'].astype(str).str.replace(' ', '')
replacements = {'.': 'UNKNOWN', 'lli': 'UNKNOWN', 'N': 'UNKNOWN'}
attacks['sex'] = attacks['sex'].replace(replacements, inplace=True)

In [17]:
# Now we want to change 'age'
# There are too many ranges now to do something

In [18]:
unique_values = attacks['fatal_boolean'].unique()
print(unique_values)

['N' 'Y' 'UNKNOWN' 'M' '2017' ' N' 'N ' 'y']


### 5. Fatal Y/N

In [19]:

# Now we want to change 'fatal_boolean'
attacks['fatal_boolean'].fillna('UNKNOWN', inplace=True)
booleans_y = {'M': 'UNKNOWN', '2017': 'UNKNOWN'}
booleans_n = {' N': 'N', 'N ': 'N'}
booleans_u = {'y': 'Y'}
attacks['fatal_boolean'] = attacks['fatal_boolean'].replace(booleans_n)
attacks['fatal_boolean'] = attacks['fatal_boolean'].replace(booleans_y)
attacks['fatal_boolean'] = attacks['fatal_boolean'].replace(booleans_u)

In [20]:
attacks['fatal_boolean'] = attacks['fatal_boolean'].replace(booleans_n)
attacks['fatal_boolean'] = attacks['fatal_boolean'].replace(booleans_y)
attacks['fatal_boolean'] = attacks['fatal_boolean'].replace(booleans_u)



### 6. Time

In [21]:
# There was too many info and i needed to make it more easy to understand so i went by steps

In [22]:
# 1.st I tried to reemplace the NaN
    # attacks['time'].fillna('Unknown', inplace=True)
# 2nd and Reemplace the values of the hours from AAhAA to AA.
    #attacks['time'] = attacks['time'].str.replace(r'\d+h\d+', 'AB', regex=True)
# 3rd i ask CHATGPT to modify the REDEX so more hours can be changed at once
    #attacks['time'] = attacks['time'].str.replace(r'(\d{2})h\d{2}', r'\1', regex=True)
# 4rd i created a for loop to include the times of the day and i asked again my friend to make it work all together

In [23]:
attacks['time'].fillna('Unknown', inplace=True)
attacks['time'] = attacks['time'].str.replace(r'(\d{2})h\d{2}', r'\1', regex=True)

def categorize_time(time_str):
    try:
        hour = int(time_str)
        if 5 <= hour < 12:
            return 'Morning'
        elif 12 <= hour < 17:   
            return 'Afternoon'
        elif 17 <= hour < 21:
            return 'Evening'
        else:
            return 'Night'
    except ValueError:
        return 'Unknown'

attacks['time'] = attacks['time'].apply(categorize_time)

In [24]:

attacks.isna().sum()


attacks.dtypes

unique_values = attacks['fatal_boolean'].unique()
print(unique_values)

['N' 'Y' 'UNKNOWN']


In [25]:
attacks.to_csv("atta_mod.csv", sep=';', encoding='utf-8')